In [15]:
from langchain import LLMMathChain, OpenAI, SerpAPIWrapper, SQLDatabase

from langchain.agents import OpenAIFunctionsAgent, initialize_agent, Tool, AgentExecutor, AgentType

 
from langchain.memory import ConversationBufferMemory, ReadOnlySharedMemory, ConversationBufferWindowMemory
from langchain.chat_models import ChatOpenAI


from langchain.prompts import MessagesPlaceholder
from langchain.schema import SystemMessage



from utils.get_credential import get_credentials, is_valid_token

import datetime

import os
import json
import xmlrpc.client



from pydantic import BaseModel, Field
from typing import List, Optional
# from utils.get_credential import get_credentials, is_valid_token


OPENAI_API_KEY = os.environ['OPENAI_KEY']
SERPAPI_API_KEY = os.environ['SERPAPI_API_KEY']
llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0613",
                 openai_api_key=OPENAI_API_KEY)

search = SerpAPIWrapper(serpapi_api_key=SERPAPI_API_KEY)
llm_math_chain = LLMMathChain.from_llm(llm=llm, verbose=True)
db = SQLDatabase.from_uri("sqlite:///user_data.db")
 


class OdooData(BaseModel):
    model: str
    fields: List[str]
    filter: List[List[str]]
    order: str
    limit: int


class OdooBrowseDataSchema(BaseModel):
    odoo_data: List[OdooData]


def OdooBrowse(data: OdooBrowseDataSchema):

    credentials_data = get_credentials(
        '47baa3066fd400e564980ff8acacca67d7436559cc1a1c49600a36364de03ac2')
    # print(f'Credentials : {credentials_data}')

    url = credentials_data['url']
    db = credentials_data['db']
    username = credentials_data['username']
    password = credentials_data['password']

    # chek uid from odoo
    common = xmlrpc.client.ServerProxy('{}/xmlrpc/2/common'.format(url))
    models = xmlrpc.client.ServerProxy('{}/xmlrpc/2/object'.format(url))

    common = xmlrpc.client.ServerProxy(f'{url}/xmlrpc/2/common'.format(db))
    uid = common.authenticate(db, username, password, {})

    odoo_data = data[0]

    model = odoo_data['model']
    fields = odoo_data['fields']
    filter_list = odoo_data['filter']
    filter = [tuple(sublist) for sublist in filter_list]

    order = odoo_data['order']
    limit = odoo_data['limit']

    params = {'fields': fields}
    if limit is not None:
        params['limit'] = limit
    if order is not None:
        params['order'] = order

    # print(f'\n\nParameters : {params}')
    # print(f'Filter : {filter}')

    run_str = f"models.execute_kw('{str(db)}', '{str(uid)}', '{str(password)}', '{str(model)}', 'search_read', [{str(filter)}], {str(params)})"
    # print(f'Command : {run_str}')

    try:
        data = eval(run_str)
        # print(f'Data : {data}')

    # data = models.execute_kw(db, uid, password, model, 'search_read', filter, params)
    except Exception as e:
        print(e)
        error_str = '{"error":' + str(e) + '}'
        # Handle the error here
        data = json.loads(error_str)

    return data




def handle_error(e):
    print(f"Error: {e}")
    return f"Error: {e}"


search = SerpAPIWrapper()

tools = [
    Tool(
        name="Search",
        func=search.run,
        description="Gunakan ketika Anda perlu menjawab pertanyaan tentang peristiwa terkini.",
        verbose=True,
        
        )
    ]

today = datetime.datetime.now().strftime("%d %B %Y")

tool = Tool(
    name="OdooBrowse",
    func=OdooBrowse,
    args_schema=OdooBrowseDataSchema,
    description=f"""Mencari data dari sistem ERP Odoo versi 12 dengan orm odoo. berikan output sesuai dengan 'args_schema'.
        Saat melakukan filter gunakan sebisa mungkin operator 'ilike' alih-alih '='.
        Hari ini adalah tanggal {today}""",
     
    verbose=True,
     
)

tools.append(tool)


memory_key = "chat_history" 
system_message = SystemMessage(
    content="Bantu sebaik mungkin untuk bisa menjawab dengan mengunakan Tools yang tersedia. Apabila ada pertanyaan yang sulit dijawab, jawab secara positif seperti halnya manusia menjawab."  
)

prompt = OpenAIFunctionsAgent.create_prompt(
    system_message=system_message,
    extra_prompt_messages=[MessagesPlaceholder(variable_name=memory_key)]  
)



memory = ConversationBufferMemory(memory_key=memory_key)

# Gunakan ReadOnlySharedMemory agar tool hanya bisa baca memory
readonlymemory = ReadOnlySharedMemory(memory=memory)

print(f'prompt : {prompt}')
print(f'memory : {memory}')
print(f'readonlymemory : {readonlymemory}')



openai_agent = initialize_agent(tools, llm, agent=AgentType.OPENAI_FUNCTIONS, verbose=True)


while True:
    incoming_message = input("You: ")
    response = openai_agent.run(incoming_message)


prompt : input_variables=['chat_history', 'agent_scratchpad', 'input'] output_parser=None partial_variables={} messages=[SystemMessage(content='Bantu sebaik mungkin untuk bisa menjawab dengan mengunakan Tools yang tersedia. Apabila ada pertanyaan yang sulit dijawab, jawab secara positif seperti halnya manusia menjawab.', additional_kwargs={}), MessagesPlaceholder(variable_name='chat_history'), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], output_parser=None, partial_variables={}, template='{input}', template_format='f-string', validate_template=True), additional_kwargs={}), MessagesPlaceholder(variable_name='agent_scratchpad')]
memory : chat_memory=ChatMessageHistory(messages=[]) output_key=None input_key=None return_messages=False human_prefix='Human' ai_prefix='AI' memory_key='chat_history'
readonlymemory : memory=ConversationBufferMemory(chat_memory=ChatMessageHistory(messages=[]), output_key=None, input_key=None, return_messages=False, human_prefix='Hu

JSONDecodeError: Expecting value: line 1 column 10 (char 9)